### Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

In [ ]:
data = pd.read_csv('data.csv')

### Описание данных

children         — количество детей в семье
days_employed    — общий трудовой стаж в днях
dob_years        — возраст клиента в годах
education        — уровень образования клиента
education_id     — идентификатор уровня образования
family_status    — семейное положение
family_status_id — идентификатор семейного положения
gender           — пол клиента
income_type      — тип занятости
debt             — имел ли задолженность по возврату кредитов
total_income     — ежемесячный доход
purpose          — цель получения кредита


Для просмотра основных и примерных значениях в дата сете выполняем функцию, которая отобразит уникальные значения по всем колонкам

In [ ]:
print(data.value_counts())

### Поиск пропусков

Поиск пропусков по столбцам, пропуски замечаны в столбцах: days_employed, total_income

In [ ]:
print(data.isnull().value_counts())

In [ ]:
children  days_employed  dob_years  education  education_id  family_status  family_status_id  gender  income_type  debt   total_income  purpose
False     False          False      False      False         False          False             False   False        False  False         False      19351
          True           False      False      False         False          False             False   False        False  True          False       2174

### Процент пропусков

Поиск процента пропущенных значений

Создание функции возвращающая процент значений Nan

In [ ]:
def skip_percentage(column):
    percentage = (column[1] / (column[1] + column[0])) * 100
    return f'{column.name}, values: Nan = {int(percentage)}%'

In [ ]:
days_employed, values: Nan = 10%
total_income, values: Nan = 10%

Не заполненые поля появились на мой взгляд из-за человеческого фактора (пользователь просто забыл завести параметры)

### Исправление пропусков

Пропущенные значению нужно заполнить медианным значением, потому что для анализа 10% не заполненых значений приемленно, так же среднее значение подходит из-за того что значение приведено в типе float

In [ ]:
def replacing_passes(column):
    # пропущенные значения будут заполнены медианой по колонке
    median_value = data[column].median()
    data[column] = data[column].fillna(median_value)
    # проверка
    print(data[column].isnull().value_counts())  
    
replacing_passes('days_employed')
replacing_passes('total_income')

### Избавленоие от артефактов

В колонке days_employed встретилось множество артефактов, трудовой стаж в днях имеет отрицательные значение, такого быть не может, предлогаем что минус попал при не правильном заполнение или переносе информации

Так как мы не знаем сколько на самом деле дней человек проработал, было принято решение возвести имеющие значения в модуль числа, что бы дни, которые было стали с положительным знаком

In [ ]:
def fixing_artifacts(column):
    # исправление артифактов (избавление от отрицательных дней)
    data[column] = abs(data[column])
    # проверка
    print(data[column].head(15))

### Изменение типа колонка 

Значение прибыли в дата сети используется вещественный тип, для дальнейшей группиризации и просто чистки данных изменим его на целочисленый

In [ ]:
def type_replacement(column, new_type, data):
    # изменение типа в колонке
    try:
        return data.astype({column: new_type})
    except:
        print('Ошибка при изменение типа')

### Дубли

В дата фрейме были замечены дубли (в колонке education), есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв 

Причины появлений из-за того что у разных пользователей не было договоренности правил о вводе данных

Приведите их к одному регистру

In [ ]:
def duplicates_work(column):
    # изменение регистра колонки

    # до изменения
    print(data[column].duplicated().value_counts())

    data[column] = data[column].str.lower()
    # проверка (после изменения)
    print(data[column].duplicated().value_counts())

В других колонках дублей замечено не было 

In [ ]:
print('Проверка остальных столбцов с строками на дубли"' + DACH * 50)
duplicates_work('family_status')
duplicates_work('gender')
duplicates_work('income_type')
duplicates_work('purpose')

### Разделение дата фреймов 

Были замечены колоноки id (education_id, family_status_id) и значения (education, family_status), принято разделить дата фрейм, в основном оставить только поля id

In [ ]:
def new_dataframe(column_id, column):
    # создание отдельных дата фреймов
    df = pd.DataFrame({column_id: data[column_id], column: data[column]})
    return df

def drop_column(column):
    # удаление колонок
    data.drop(columns=[column], axis=1, inplace=True)
    return f'Колонка {column} удалена'


print('Создание отдельных dataframe"' + DACH * 50)
df_education = new_dataframe('education_id', 'education')
print(df_education.head(15))
print(drop_column('education'))

df_family_status = new_dataframe('family_status_id', 'family_status')
print(df_family_status.head(15))
print(drop_column('family_status'))

print(data.dtypes)

### Разделение поля прибыль на категории 

In [ ]:
def income_categories(value):
    # категории по прибыли
    if value >= 1000001:
       return 'A'
    elif 200001 <= value <= 1000000:
        return 'B'
    elif 50001 <= value <= 200000:
        return 'C'
    elif 30001 <= value <= 50000:
        return 'D'
    elif 0 < value <= 30000:
        return 'E'
    else:
        print('Нет подходящей категории для ', value)
        
        
print('Категории по приболи' + DACH * 50)
data['total_income_category'] = data['total_income'].apply(income_categories)
print(data[['total_income', 'total_income_category']].head(15))

### Разделение поля назначение на категории 

In [ ]:
def purpose_categories(value):
    # категории по назначению
    if 'авто' in value:
        return 'операции с автомобилем'
    elif 'недвиж' or 'жиль' in value:
        return 'операции с недвижимостью'
    elif 'свадьб' in value:
        return 'проведение свадьбы'
    elif 'образов' in value:
        return 'получение образования'
    else:
        print('Нет подходящей категории для ', value)

print('Категории по назначению' + DACH * 50)
data['purpose_category'] = data['purpose'].apply(purpose_categories)
print(data[['purpose', 'purpose_category']].head(15))

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Зависимости сильной нет, но 

In [ ]:
print(data[['children', 'debt']].value_counts())

children  debt
 0        0       13086
 1        0        4374
 2        0        1861
 0        1        1063
 1        1         444
 3        0         303
 2        1         194
 20       0          68
-1        0          46
 4        0          37
 3        1          27
 5        0           9
 20       1           8
 4        1           4
-1        1           1

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

#### Как разные цели кредита влияют на его возврат в срок?